<a target="_blank" href="https://colab.research.google.com/github/tatsath/Interpretability/blob/main/ActivationDetectTwoPass.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [89]:
!pip install nnsight==0.4.0 sae-lens

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [90]:
!pip install --upgrade --force-reinstall numpy scipy scikit-learn

  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (37.6 MB)
Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.5 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 3.6.0
    Uninstalling threadpoolctl-3.6.0:
      Successfully uni

In [91]:
!pip install neuronpedia

In [92]:
#!pip install --upgrade --force-reinstall numpy

In [93]:
# Import necessary libraries
from nnsight import LanguageModel
import torch
import pandas as pd

import huggingface_hub


# Initialize the language model with specific parameters
model = LanguageModel("google/gemma-2b-it", trust_remote_code=True, device_map="cuda:0", low_cpu_mem_usage=True, torch_dtype=torch.float16)
model.requires_grad_(False)



from sae_lens import SAE
layer = 6

# get the SAE for this layer
sae, cfg_dict, _ = SAE.from_pretrained(
    release = "gemma-2b-res-jb",
    sae_id = f"blocks.{layer}.hook_resid_post",
    device = 'cuda:0'
)
# Load the SAE weights using safetensors
# from safetensors import safe_open
# with safe_open("/home/ashater/work/sae.safetensors", framework="pt") as st:
#     w_dec = st.get_tensor("W_dec").to('cuda:1')





In [94]:

from huggingface_hub import hf_hub_download

# Define the repository ID and filename
repo_id = "jbloom/Gemma-2b-Residual-Stream-SAEs"
filename = "sae_weights.safetensors"

# Download the file


# filepath = hf_hub_download(repo_id=repo_id, filename=filename, subfolder="gemma_2b_blocks.6.hook_resid_post_16384")

#Layer 12
#filepath = hf_hub_download(repo_id=repo_id, filename=filename, subfolder="gemma_2b_blocks.12.hook_resid_post_16384")

# Layer 6
filepath = hf_hub_download(repo_id=repo_id, filename=filename, subfolder="gemma_2b_blocks.6.hook_resid_post_16384_anthropic_fast_lr")

# Print the downloaded file path
print(filepath)

# Print the downloaded file path
print(filepath)

/root/.cache/huggingface/hub/models--jbloom--Gemma-2b-Residual-Stream-SAEs/snapshots/2e64e9127f413318e0500a6ba3981483f251bcee/gemma_2b_blocks.6.hook_resid_post_16384_anthropic_fast_lr/sae_weights.safetensors
/root/.cache/huggingface/hub/models--jbloom--Gemma-2b-Residual-Stream-SAEs/snapshots/2e64e9127f413318e0500a6ba3981483f251bcee/gemma_2b_blocks.6.hook_resid_post_16384_anthropic_fast_lr/sae_weights.safetensors


In [95]:
# Load the SAE weights using safetensors
from safetensors import safe_open
with safe_open(filepath, framework="pt") as st:
    w_dec = st.get_tensor("W_dec").to('cuda:0')

In [96]:
import torch
#torch.cuda.is_available()

#torch.cuda.device_count()
#len(docs)
torch.cuda.set_device(0)
torch.cuda.current_device()


0

In [97]:
#sae.W_dec == w_dec

cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
cos(w_dec[471,:], sae.W_dec).topk(100).indices

tensor([  471,  4579, 11129, 11728,  6446,  6510, 14611,  7687,  5411,   891,
        15780,  7372,   675, 13371,   713, 15976,  1076,  5419,   976, 13837,
        10036,  2817,  6495,   154,  2287, 15131, 16291,  3466,  2111, 14557,
         9715,  8481,  8099,  8567, 15023, 11584,  3869,  7062, 15369,  3864,
        14291,  5239,  7807,  4189,  4815, 13685, 10031,  2126,  5936, 14224,
        15698, 14444, 10105, 10185,  5407,  6545,  8586,  8753,  6027, 13939,
        11269,  9963,  8461, 15760,  2312,   544,  9188, 11808,  8498,   354,
         4989,  7682, 11355,  4828, 14022,  3586,  8306,   954, 12170,   167,
        11255,  9691,  6426, 11718, 14749,  7608, 14153,  3971, 14416, 13584,
         3900, 13970,  8222, 14874,  8523,  9004, 12520, 15384,  1805,   575],
       device='cuda:0')

Parameters

In [98]:
#@title Self-explanation in 18 lines
feature = 471  #@param {type: "integer"}
#feature = 5892  #@param {type: "integer"}
# feature = 9415
# feature = 3629
scale = 1 #@param {type: "number"}
se_demo = True  #@param {type: "boolean"}
max_new_tokens = 200  #@param {type: "integer"}
n_generate = 1  #@param {type: "integer"}
# vals = ['positive', 'neutral', 'negative']
# samples = pd.read_excel('/home/ashater/work/sae/sentence_sentiment_gpt4o.xlsx')

## Main code prompt


In [99]:
full_res = []

prompt = "Credit Risk related"
# prompt = """Credit Risk Related"""




with model.generate(prompt, max_new_tokens=max_new_tokens, num_return_sequences=n_generate, do_sample=True, scan=False, validate=False) as gen:
    # get final output saved
    out = model.generator.output.save()
    # capture at layer 6 or 12
    resid = model.model.layers[6].output.save()

features = sae.encode(resid[0])

summed_activations = features.abs().sum(dim=1) # Sort by max activations

print(summed_activations)
summed_activations = features
top_activations_indices = summed_activations.topk(1000) # Get indices of top 20
print(top_activations_indices)
#top_activations_indices = summed_activations.indices # Get indices of top 20


for i, l in enumerate(model.tokenizer.batch_decode(out)):
    #print(model.tokenizer.batch_decode(out))
    s = repr(l.partition(prompt)[2].partition("<eos>")[0])
    #print(prompt + "|"+ s + "|" + str(top_activations_indices))
    #print(prompt + "|"+ l +"\n" + "|" + str(top_activations_indices))
    #print(str(top_activations_indices))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tensor([[ 2.6458, 19.0831,  0.0000,  ...,  0.0000, 11.5059,  0.0000]],
       device='cuda:0', grad_fn=<SumBackward1>)
torch.return_types.topk(
values=tensor([[[51.1168, 48.4639, 45.8360,  ..., 18.9703, 18.9648, 18.9466],
         [13.9223,  3.4922,  2.9306,  ...,  0.0000,  0.0000,  0.0000],
         [12.2706,  5.2049,  2.2141,  ...,  0.0000,  0.0000,  0.0000],
         [11.3995,  4.3096,  2.9391,  ...,  0.0000,  0.0000,  0.0000]]],
       device='cuda:0', grad_fn=<TopkBackward0>),
indices=tensor([[[ 3390, 15881,  5347,  ...,  8207,    15,  3380],
         [ 5419, 10035,   471,  ...,   769,   770,   771],
         [10870,   471, 12704,  ...,   899,   900,   901],
         [15570,  4633, 11912,  ...,   889,   890,   891]]], device='cuda:0'))


## summed activations across tokens, finance features

In [100]:
s

'" to Corporate Bonds\\n\\nWhat is credit risk related to corporate bonds?\\n\\nCredit risk is the risk that an issuer will not make interest or principal payments on a corporate bond when due.\\n\\nWhat factors contribute to credit risk?\\n\\n* Financial risk: A decline in the issuer\'s financial health, such as debt defaults, loan defaults, and high interest payments.\\n* Political risk: Government actions that may impact the economy, interest rates, and corporate profits.\\n* Industry risk: Certain industries are more vulnerable to cyclical fluctuations and economic downturns.\\n* Corporate governance risk: Weak corporate governance practices can lead to financial mismanagement and debt defaults.\\n* Market risk: Factors such as interest rate changes, inflation, and volatility can affect bond prices.\\n\\nHow does credit risk affect investors?\\n\\n* Reduced returns: Investors may not earn as much interest or principal payments on bonds issued by companies with high credit risk.\\n*

In [101]:
 #model.tokenizer.decode(model.tokenizer.encode(prompt)[2])
summed_activations[0][0]


tensor([ 2.6458, 19.0831,  0.0000,  ...,  0.0000, 11.5059,  0.0000],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [102]:
#summed_activations[summed_activations > 10].shape
idc = summed_activations.topk(1000).indices
[g in idc for g in [354, 2105, 4218, 5239, 6545, 7529, 8461, 11614, 11916, 14291, 15399]]

[True, False, False, False, False, False, True, False, False, False, False]

In [103]:
#summed_activations[summed_activations > 10].shape
idc = summed_activations.topk(1139).indices
[g in idc for g in [10870]]

[True]

## Get features for each token

In [104]:
idc = summed_activations.mean(dim=1).topk(5000).indices
[g in idc for g in [354, 2105, 4218, 5239, 6545, 7529, 8461, 11614, 11916, 14291, 15399]]

[False, False, False, False, True, False, False, True, False, False, True]

In [105]:
prompt = "Credit Risk"
# prompt = """Credit Risk Related"""



#prompt = '<start_of_turn>user\nWhat is the meaning of the word "X"?<end_of_turn>\n<start_of_turn>model\nThe meaning of the word "X" is "'

# prompt = '<start_of_turn>user\n' + ismpl +  '<end_of_turn>\n<start_of_turn>model X'
# positions = [i for i, a in enumerate(model.tokenizer.encode(prompt)) if model.tokenizer.decode([a]) == "X"]
# print(positions)
#with model.generate(prompt, max_new_tokens=max_new_tokens, num_return_sequences=n_generate, do_sample=False, scan=False, validate=False, temperature=0.0) as gen:
with model.generate(s, max_new_tokens=max_new_tokens, num_return_sequences=n_generate, do_sample=True, scan=False, validate=False) as gen:
    # vector steering section
    # for feature_i in topics:
    # vector = sae.W_dec[[feature]]
    # vector = vector / vector.norm()
    # vector = vector * scale
    # setting vector into layer 2
    # for position in positions:
    #   model.model.layers[2].output[0][:, position] = vector

    # get final output saved
    out = model.generator.output.save()
    # capture at layer 6 or 12
    resid = model.model.layers[6].output.save()

features = sae.encode(resid[0])

summed_activations = features.abs().sum(dim=1) # Sort by max activations

print(summed_activations)
summed_activations = features
top_activations_indices = summed_activations.topk(1000) # Get indices of top 20
print(top_activations_indices)
#top_activations_indices = summed_activations.indices # Get indices of top 20


for i, l in enumerate(model.tokenizer.batch_decode(out)):
    #print(model.tokenizer.batch_decode(out))
    s = repr(l.partition(prompt)[2].partition("<eos>")[0])
    #print(prompt + "|"+ s + "|" + str(top_activations_indices))
    #print(prompt + "|"+ l +"\n" + "|" + str(top_activations_indices))
    #print(str(top_activations_indices))

tensor([[ 2.6473, 19.0836,  0.0000,  ...,  0.0000, 11.5040,  0.0000]],
       device='cuda:0', grad_fn=<SumBackward1>)
torch.return_types.topk(
values=tensor([[[51.1153, 48.4625, 45.8345,  ..., 18.9746, 18.9655, 18.9496],
         [ 5.7439,  3.7488,  2.1953,  ...,  0.0000,  0.0000,  0.0000],
         [ 5.1796,  3.8137,  3.1580,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 5.1268,  2.7663,  1.4892,  ...,  0.0000,  0.0000,  0.0000],
         [ 3.0255,  2.9250,  2.8798,  ...,  0.0000,  0.0000,  0.0000],
         [ 6.6035,  1.9238,  1.9180,  ...,  0.0000,  0.0000,  0.0000]]],
       device='cuda:0', grad_fn=<TopkBackward0>),
indices=tensor([[[ 3390, 15881,  5347,  ...,  8207,    15,  3380],
         [ 2307,  2745,  3586,  ...,   943,   944,   945],
         [ 3558,  5145,  7764,  ...,   946,   947,   948],
         ...,
         [  562, 13825, 14611,  ...,   973,   974,   975],
         [14261, 11672,  5862,  ...,   882,   883,   884],
         [ 5145,  2608,  8153,  ...,   9

In [106]:
summed_activations.mean(dim=1).topk(100)

torch.return_types.topk(
values=tensor([[1.1994, 1.1449, 1.0269, 0.8518, 0.6048, 0.4731, 0.4102, 0.3808, 0.3791,
         0.3747, 0.3601, 0.3381, 0.3075, 0.3031, 0.2708, 0.2649, 0.2581, 0.2522,
         0.2512, 0.2497, 0.2492, 0.2383, 0.2366, 0.2244, 0.2198, 0.2151, 0.2122,
         0.2119, 0.2092, 0.2090, 0.2082, 0.2081, 0.2079, 0.2054, 0.2050, 0.2034,
         0.2022, 0.2006, 0.1987, 0.1945, 0.1932, 0.1924, 0.1923, 0.1882, 0.1882,
         0.1875, 0.1871, 0.1866, 0.1862, 0.1857, 0.1853, 0.1849, 0.1844, 0.1844,
         0.1815, 0.1808, 0.1802, 0.1791, 0.1790, 0.1784, 0.1777, 0.1774, 0.1767,
         0.1758, 0.1747, 0.1746, 0.1743, 0.1734, 0.1732, 0.1731, 0.1728, 0.1722,
         0.1713, 0.1712, 0.1709, 0.1704, 0.1704, 0.1701, 0.1693, 0.1691, 0.1691,
         0.1688, 0.1677, 0.1676, 0.1675, 0.1673, 0.1672, 0.1666, 0.1662, 0.1651,
         0.1649, 0.1648, 0.1647, 0.1647, 0.1643, 0.1643, 0.1642, 0.1637, 0.1637,
         0.1636]], device='cuda:0', grad_fn=<TopkBackward0>),
indices=tensor(

In [107]:
summed_activations.sum(dim=1).topk(100)

torch.return_types.topk(
values=tensor([[259.0632, 247.2945, 221.8063, 183.9945, 130.6425, 102.1969,  88.6068,
          82.2445,  81.8860,  80.9394,  77.7916,  73.0346,  66.4262,  65.4611,
          58.4914,  57.2112,  55.7461,  54.4838,  54.2522,  53.9337,  53.8233,
          51.4691,  51.1153,  48.4625,  47.4828,  46.4595,  45.8345,  45.7785,
          45.1872,  45.1530,  44.9711,  44.9553,  44.8998,  44.3714,  44.2888,
          43.9322,  43.6733,  43.3399,  42.9102,  42.0151,  41.7283,  41.5674,
          41.5437,  40.6578,  40.6557,  40.4939,  40.4169,  40.2952,  40.2250,
          40.1170,  40.0223,  39.9490,  39.8380,  39.8316,  39.1949,  39.0604,
          38.9140,  38.6872,  38.6540,  38.5242,  38.3855,  38.3120,  38.1646,
          37.9701,  37.7297,  37.7093,  37.6499,  37.4591,  37.4124,  37.3915,
          37.3326,  37.1856,  37.0094,  36.9817,  36.9161,  36.8127,  36.7997,
          36.7331,  36.5778,  36.5280,  36.5211,  36.4701,  36.2337,  36.2014,
          36.1818,  

In [108]:
import pandas as pd
import torch

def get_top_activations(model, sae, prompt, max_new_tokens=200, n_generate=1, top_n=2):
  """
  Returns the top n activations in a Pandas DataFrame for both the prompt and the output.

  Args:
    model: The language model object.
    sae: The SAE object.
    prompt: The input prompt.
    max_new_tokens: The maximum number of new tokens to generate for the output.
    n_generate: The number of output sequences to generate.
    top_n: The number of top activations to return.

  Returns:
    A tuple containing two Pandas DataFrames: one for the prompt and one for the output.
  """

  # # Get activations for the prompt
  # with model.generate(prompt, max_new_tokens=max_new_tokens, num_return_sequences=n_generate, do_sample=True, scan=False, validate=False) as gen:
  #     # get final output saved
  #     out = model.generator.output.save()
  #     # capture at layer 6 or 12
  #     resid = model.model.layers[6].output.save()

  # prompt_features = sae.encode(resid[0])

  # prompt_summed_activations = features.abs().sum(dim=1) # Sort by max activations

  # print(summed_activations)
  # summed_activations = features
  # top_activations_indices = summed_activations.topk(1000) # Get indices of top 20



  with model.generate(prompt, max_new_tokens=max_new_tokens, num_return_sequences=n_generate, do_sample=True, scan=False, validate=False) as gen:
      # get final output saved
      out = model.generator.output.save()
      # capture at layer 6 or 12
      resid = model.model.layers[6].output.save()

  prompt_features = sae.encode(resid[0])

  prompt_summed_activations_topk  = prompt_features.abs().sum(dim=1).topk(top_n) # Sort by max activations
  prompt_mean_activations_topk  = prompt_features.abs().mean(dim=1).topk(top_n) # Sort by max activations
  #print(prompt_summed_activations_topk)
  #print(prompt_mean_activations_topk )



  for i, l in enumerate(model.tokenizer.batch_decode(out)):
      s = repr(l.partition(prompt)[2].partition("<eos>")[0])


  # Get activations for the output
  with model.generate(s, max_new_tokens=max_new_tokens, num_return_sequences=n_generate, do_sample=True, scan=False, validate=False) as gen:
      # out = model.generator.output.save()
      resid = model.model.layers[6].output.save()  # Capture at layer 6

  output_features = sae.encode(resid[0])
  output_summed_activations_topk = output_features.abs().sum(dim=1).topk(top_n) # Sort by max activations
  output_mean_activations_topk  = output_features.abs().mean(dim=1).topk(top_n) # Sort by max activations
  #print(output_summed_activations_topk)
  #print(output_mean_activations_topk )

  # Create prompt_df
  prompt_df = pd.DataFrame({
      "feature": prompt_summed_activations_topk.indices[0].cpu().detach().numpy(), # Added .detach()
      "sum_activation": prompt_summed_activations_topk.values[0].cpu().detach().numpy(), # Added .detach()
      "mean_activation": prompt_mean_activations_topk.values[0].cpu().detach().numpy() # Added .detach()
  })

  # Create output_df
  output_df = pd.DataFrame({
      "feature": output_summed_activations_topk.indices[0].cpu().detach().numpy(), # Added .detach()
      "sum_activation": output_summed_activations_topk.values[0].cpu().detach().numpy(), # Added .detach()
      "mean_activation": output_mean_activations_topk.values[0].cpu().detach().numpy() # Added .detach()
  })

  return prompt_df, output_df

In [109]:
# Get the top 100 activations for both prompt and output

prompt = "credit Risk related"
prompt_df, output_df = get_top_activations(model, sae, prompt, top_n=10)

# Display the DataFrames
print("Prompt activations:")
display(prompt_df)

print("\nOutput activations:")
display(output_df)

Prompt activations:


,feature,sum_activation,mean_activation
0,3390,51.116844,12.779211
1,15881,48.463902,12.115975
2,5347,45.836033,11.459008
3,1630,45.771412,11.442853
4,16334,45.188702,11.297175
5,556,45.154243,11.288561
6,5490,44.291336,11.072834
7,11785,43.933575,10.983394
8,1587,43.676056,10.919014
9,1491,41.534500,10.383625



Output activations:


,feature,sum_activation,mean_activation
0,3553,194.987213,0.928511
1,4675,150.807617,0.718132
2,10035,142.283264,0.677539
3,3464,128.369049,0.611281
4,12704,100.131165,0.476815
5,9927,99.097427,0.471893
6,5419,82.754883,0.394071
7,2788,77.473770,0.368923
8,5862,76.070160,0.362239
9,9415,75.968262,0.361754


Bias Use case

In [128]:
# Get the top 100 activations for both prompt and output

prompt = "Credit Risk Related"
prompt_df, output_df = get_top_activations(model, sae, prompt, top_n=5)

# Display the DataFrames
print("Prompt activations:")
display(prompt_df)

print("\nOutput activations:")
display(output_df)

Prompt activations:


,feature,sum_activation,mean_activation
0,3390,51.116844,12.779211
1,15881,48.463902,12.115975
2,5347,45.836033,11.459008
3,1630,45.771412,11.442853
4,16334,45.188702,11.297175



Output activations:


,feature,sum_activation,mean_activation
0,9927,172.218567,0.762029
1,3553,153.923248,0.681076
2,471,124.793152,0.552182
3,10870,97.573593,0.431742
4,15857,95.156395,0.421046


## Autointerp from Neuronpedia

In [129]:
import requests
import json
from google.colab import userdata

def get_or_generate_activation_description(modelId, layer, index, apiKey):
    """Retrieves or generates an activation description.

    Args:
        modelId: The model ID.
        layer: The layer number.
        index: The neuron index.
        apiKey: Your Neuronpedia API key.

    Returns:
        The activation description, or None if not found.
    """

    # Try to retrieve the description
    retrieve_url = f"https://neuronpedia.org/api/feature/{modelId}/{layer}/{index}"
    retrieve_response = requests.get(retrieve_url)
    retrieve_data = retrieve_response.json()
    description = find_description(retrieve_data)  # Assuming find_description is defined

    # If description is not found, generate it
    #print(description)
    if description is None:
        generate_url = "https://www.neuronpedia.org/api/explanation/generate"
        payload = {
            "modelId": modelId,
            "layer": layer,
            "index": index,
            "explanationType": "oai_attention-head",
            "explanationModelName": "gpt-4o"
        }
        headers = {
            "Content-Type": "application/json",
            "x-api-key": apiKey
        }
        generate_response = requests.post(generate_url, json=payload, headers=headers)
        generate_data = generate_response.json()
        print(generate_data)
        description = generate_data.get('explanation', {}).get('description', None)

    return description

def find_description(data):
    if 'explanations' in data:
        for item in data['explanations']:
            return item['description']
    # recursively check nested dictionaries
    for key, value in data.items():
        if isinstance(value, dict):
            result = find_description(value)
            if result:
                return result
        elif isinstance(value, list):
            for element in value:
                if isinstance(element, dict):
                    result = find_description(element)
                    if result:
                        return result
    return None


# Example usage:
apiKey = userdata.get('NEURONPEDIA_API_KEY')  # Replace with your actual API key
description = get_or_generate_activation_description("gemma-2b", "6-res-jb", "10006", apiKey)
print("Description:", description)

Description: words related to specific fruits, especially cherry


In [137]:
# ... (your previous code to define get_or_generate_activation_description and find_description) ...

# Assuming you have output_df and prompt_df DataFrames

# API key, modelId, and layer (replace with your actual values)

from IPython.display import display, HTML

apiKey = userdata.get('NEURONPEDIA_API_KEY')
modelId = "gemma-2b"
layer = "6-res-jb"

# Function to get feature label (same as before)
def get_feature_label(row):
    #print(int(row['feature']))
    feature_index = int(row['feature'])
    description = get_or_generate_activation_description(modelId, layer, feature_index, apiKey)
    return description

# Apply to output_df
output_df['feature_label'] = output_df.apply(get_feature_label, axis=1)

# Apply to prompt_df
prompt_df['feature_label'] = prompt_df.apply(get_feature_label, axis=1)

# Display the DataFrames
print("Prompt activations:")
display(prompt_df)

print("\nOutput activations:")
display(output_df)

Prompt activations:


,feature,sum_activation,mean_activation,feature_label
0,3390,51.116848,17.038950,the beginning of a piece of text or a document
1,15881,48.464008,16.154671,email addresses and computer-related information
2,5347,45.836063,15.278688,the beginning of blog posts or articles
3,1630,45.770149,15.256717,"information related to legal terms, company po..."
4,16334,45.188644,15.062881,definitions of acronyms



Output activations:


,feature,sum_activation,mean_activation,feature_label
0,3553,359.896362,1.689654,research papers related to health and medicine
1,14266,192.969360,0.905959,phrases related to discrimination based on var...
2,6165,84.226173,0.395428,cryptographic terms or identifiers
3,9415,76.173691,0.357623,"references to a specific organization, particu..."
4,12606,70.361496,0.330336,"technical features related to devices, equipme..."


In [134]:
def add_label(output_df, prompt_df, apiKey, modelId, layer):
    """Applies feature labels, styles, and displays the dataframes."""

    def get_feature_label(row):
        feature_index = int(row['feature'])
        description = get_or_generate_activation_description(modelId, layer, feature_index, apiKey)
        return description

    # Apply to output_df
    output_df['feature_label'] = output_df.apply(get_feature_label, axis=1)

    # Apply to prompt_df
    prompt_df['feature_label'] = prompt_df.apply(get_feature_label, axis=1)

    return output_df, prompt_df

## Bias detection


In [141]:
prompt = "Gender Bias related"
apiKey = userdata.get('NEURONPEDIA_API_KEY')
modelId = "gemma-2b"
layer = "6-res-jb"
top_n = 10
prompt_df, output_df = get_top_activations(model, sae, prompt, top_n=top_n)
prompt_df_updated, output_df_updated = add_label(output_df, prompt_df, apiKey, modelId, layer)


In [143]:
# Display the DataFrames
pd.set_option('display.max_colwidth', None)
print("Prompt activations:")
display(prompt_df_updated)

# print("\nOutput activations:")
# display(output_df_updated)

Prompt activations:


,feature,sum_activation,mean_activation,feature_label
0,3553,241.878036,1.104466,research papers related to health and medicine
1,8165,150.341843,0.686492,lies and deception in historical and political contexts
2,616,133.163513,0.608053,"criteria for non-discrimination and equal opportunities based on traits like race, gender, sexual orientation, and disability"
3,7222,124.142342,0.566860,"mentions of dates and events related to women's initiatives, such as International Women's Day"
4,14266,91.381371,0.417267,"phrases related to discrimination based on various factors, including race, religion, gender, and disability, as well as actions taken to address and prevent such discrimination"
5,3098,89.878769,0.410405,text related to online comments moderation
6,9464,88.811089,0.405530,technical information and references related to information science
7,6570,85.244171,0.389243,phrases related to news reporting and journalism
8,12757,83.486336,0.381216,misrepresented and misspelled words or phrases
9,13640,76.052254,0.347271,information related to media or journalism
